## Import packages

#### We adopt surprise pkg for recommerder system to evaluate the model. The surprise is built for analyzing recommender systems that deal with rating data

In [1]:
#pip install scikit-surprise

In [2]:
import numpy as np
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

## Load Data

In [3]:
anime = pd.read_csv('D:/McGill/INSY 695-ADV. Topic in Information Syst/Individual Assignment/Data/anime/anime.csv',
                    low_memory=False)
rating = pd.read_csv('D:/McGill/INSY 695-ADV. Topic in Information Syst/Individual Assignment/Data/anime/rating.csv',
                     low_memory=False)

In [4]:
anime = anime.rename(columns={"rating":"anime_rating"})
anime.head()

,anime_id,name,genre,type,episodes,anime_rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [5]:
df_title=anime.drop(['genre','type','episodes','anime_rating','members'], axis=1)
df_title.set_index('anime_id',inplace=True)
df_title.head(10)

,name
anime_id,
32281,Kimi no Na wa.
5114,Fullmetal Alchemist: Brotherhood
28977,Gintama°
9253,Steins;Gate
9969,Gintama&#039;
32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...
11061,Hunter x Hunter (2011)
820,Ginga Eiyuu Densetsu
15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...


In [6]:
rating = rating.rename(columns={"rating":"user_rating"})
rating.head()

,user_id,anime_id,user_rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [18]:
#merge the 2 datasets
df = rating.merge(anime, on = "anime_id", how="right")
#To reduce the size of the dataset and maintain the full rating of each user, 
#we reduce the size by selecting users with the user id <=10000
df =df[df['user_id']<=10000]
print(df.head())
print("---------------------------")
print(df.shape)

   user_id  anime_id  user_rating    name  \
0      1.0        20         -1.0  Naruto   
1      3.0        20          8.0  Naruto   
2      5.0        20          6.0  Naruto   
3      6.0        20         -1.0  Naruto   
4     10.0        20         -1.0  Naruto   

                                               genre type episodes  \
0  Action, Comedy, Martial Arts, Shounen, Super P...   TV      220   
1  Action, Comedy, Martial Arts, Shounen, Super P...   TV      220   
2  Action, Comedy, Martial Arts, Shounen, Super P...   TV      220   
3  Action, Comedy, Martial Arts, Shounen, Super P...   TV      220   
4  Action, Comedy, Martial Arts, Shounen, Super P...   TV      220   

   anime_rating  members  
0          7.81   683297  
1          7.81   683297  
2          7.81   683297  
3          7.81   683297  
4          7.81   683297  
---------------------------
(1042697, 9)


In [19]:
df_p = pd.pivot_table(df,
                      values='user_rating',
                      index='user_id',
                      columns='anime_id')

print(df_p.shape)

(10000, 8337)


In [20]:
df_p.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,34048,34085,34103,34107,34136,34173,34240,34283,34324,34325
user_id,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,8.0,NaN,NaN,6.0,NaN,6.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['user_id', 'anime_id', 'user_rating']], reader)

svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    3.3078  3.3024  3.3050  3.3051  0.0022  
MAE (testset)     2.9692  2.9630  2.9649  2.9657  0.0026  
Fit time          28.79   29.14   29.04   28.99   0.15    
Test time         2.52    2.26    2.56    2.45    0.13    


{'test_rmse': array([3.30778261, 3.30237589, 3.30499931]),
 'test_mae': array([2.9691672 , 2.96298172, 2.96491961]),
 'fit_time': (28.793102741241455, 29.143804788589478, 29.042835474014282),
 'test_time': (2.524871349334717, 2.2606186866760254, 2.561729669570923)}

### Recommend using correlation

In [22]:
def anime_recom(title):
    print("The related animes to ({}):".format(title))
    i = int(df_title.index[df_title['name'] == title][0])
    target=df_p[i]
    corr=df_p.corrwith(target)
    corr_target = pd.DataFrame(corr, columns = ['Correlation'])
    corr_target.dropna(inplace = True)
    corr_target = corr_target.sort_values('Correlation', ascending = False)
    corr_target.index = corr_target.index.map(int)
    corr_target = corr_target.join(df_title)
    print(corr_target[:10].to_string(index=False))
    print(corr_target)
    print(corr_target['Correlation'].tolist())

In [23]:
anime_recom("Kiseijuu: Sei no Kakuritsu")

The related animes to (Kiseijuu: Sei no Kakuritsu):


D:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
D:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


 Correlation                                   name
         1.0                           Chiryokumaru
         1.0                             Outlanders
         1.0                       Toei Robot Girls
         1.0                Shoukoujo The Animation
         1.0                     Stratos 4: Logbook
         1.0                      Plawres Sanshirou
         1.0                            Coluboccoro
         1.0                               Dokkoida
         1.0  Toriko: Barbarian Ivy wo Hokaku Seyo!
         1.0                                 School
          Correlation                                            name
anime_id                                                             
34324             1.0                                    Chiryokumaru
1940              1.0                                      Outlanders
10954             1.0                                Toei Robot Girls
11065             1.0                         Shoukoujo The Animation
1903    

In [ ]:
anime_recom("Kiseijuu: Sei no Kakuritsu")

In [ ]:
df_title.index[df_title['name'] == 'Howl no Ugoku Shiro'][0]


### To reduce the run time, we sample a portion of the original dataset

In [ ]:
i = int(df_title.index[df_title['name'] == 'i = int(df_title.index[df_title['name'] == title][0])'][0])
target=df_p[i]
corr=df_p.corrwith(target)

In [ ]:
corr=df_p.corrwith(target)

In [ ]:
data_reduced = data.sample(frac=0.1,random_state=1)
data_reduced.shape